Basic structure of the code is similar to 'delta_distribution_analysis.ipynb'
What we are going to do is re analyze the performance of CSCS algorithm, removing the infulence of D614G mutation since the effect of this mutation is too large compared to other mutations.

To achieve this, we changed the input wild type to 'D614G' applied one and re-inferred the CSCS model with the input.

In [20]:
"""Load the result of D614G_wt inferred result."""
import pandas as pd
file_name = "analyze_semantics_d614g_bilstm_512.txt"
# file_name = "extended_analyze_semantics_cov_bilstm_512.txt"
d614g_df = pd.read_csv(file_name, sep='\t')
d614g_df

,pos,wt,mut,prob,change,is_viable,is_escape
0,0,M,A,2.189499e-06,1319.03750,True,False
1,0,M,B,1.117368e-18,1573.24770,False,False
2,0,M,C,1.520572e-06,3205.32890,True,False
3,0,M,D,6.675085e-09,1647.54540,True,False
4,0,M,E,1.990659e-07,1503.97990,True,False
...,...,...,...,...,...,...,...
30547,0,M,V,8.088458e-06,737.07324,True,False
30548,0,M,W,1.014628e-09,1087.49430,True,False
30549,0,M,X,1.909755e-11,395.38600,False,False
30550,0,M,Y,2.780735e-07,554.93100,True,False


In [21]:
# check whether 'is_escape' label is marked
d614g_df[d614g_df['is_escape'] == True]
# Result saids these are marked.

,pos,wt,mut,prob,change,is_viable,is_escape
4933,416,K,E,3.921576e-05,2844.8367,True,True
5227,444,V,A,1.768988e-04,4546.4453,True,True
5298,449,N,D,2.713998e-05,2976.5283,True,True
5347,452,Y,F,6.670675e-04,2534.2595,True,True
5386,454,L,F,1.124983e-04,6083.0503,True,True
5712,484,G,D,3.269176e-04,3935.3152,True,True
5798,492,Q,K,5.951609e-07,4014.0322,True,True
8030,681,R,Q,9.068682e-05,4047.4768,True,True
8086,686,V,G,6.209489e-07,4869.4050,True,True
9078,768,G,E,3.736911e-04,4598.4116,True,True


In [22]:
"""Make rank of 'prob', 'change'"""
rank_names = ['grammar rank', 'sem change rank']
score_names = ['prob', 'change']
def rank_helper(df, rank_names, score_names):
    df[rank_names] = df[score_names].rank(method="min", ascending=False)

rank_helper(d614g_df, rank_names, score_names)
d614g_df

,pos,wt,mut,prob,change,is_viable,is_escape,grammar rank,sem change rank
0,0,M,A,2.189499e-06,1319.03750,True,False,9918.0,29349.0
1,0,M,B,1.117368e-18,1573.24770,False,False,30548.0,28351.0
2,0,M,C,1.520572e-06,3205.32890,True,False,10771.0,15029.0
3,0,M,D,6.675085e-09,1647.54540,True,False,20671.0,27996.0
4,0,M,E,1.990659e-07,1503.97990,True,False,15310.0,28646.0
...,...,...,...,...,...,...,...,...,...
30547,0,M,V,8.088458e-06,737.07324,True,False,6822.0,30465.0
30548,0,M,W,1.014628e-09,1087.49430,True,False,22373.0,30021.0
30549,0,M,X,1.909755e-11,395.38600,False,False,25295.0,30548.0
30550,0,M,Y,2.780735e-07,554.93100,True,False,14611.0,30515.0


In [23]:
"""
Add up grammar and semantic rank.
Check the mean and std of add up result of escape mutants.
"""
d614g_df['cscs score'] = d614g_df['grammar rank'] + d614g_df['sem change rank']
d614g_df['cscs rank'] = d614g_df['cscs score'].rank(method="min")
d614g_df_escapes = d614g_df[d614g_df['is_escape'] == True]
d614g_df_escapes['cscs rank'].mean()

4128.894736842105